In [1]:
import os
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
from queries import Simbad
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import itertools
% matplotlib inline
x1d_dir='/DataDisk/datafiles/PGCC_HST/x1d_files/'

# Returns the velocity shift necessary to convert Helio to LSR
# i.e. returns (V_lsr - V_helio)
def HelioToLSR(target_ra_deg, target_dec_deg):
    Vsun=19.7
    a0=271.0
    d0=30.0
    shift=Vsun*(np.cos(target_ra_deg-a0)*np.cos(a0)*np.cos(target_ra_deg)+np.sin(d0)*np.sin(target_dec_deg))
    return shift


#Returns angular separation between two Ra/Dec coordinates in degrees
def get_ang_sep(ra1,dec1,ra2,dec2):
    ra1_rad=ra1*np.pi/180.
    dec1_rad=dec1*np.pi/180.
    ra2_rad=ra2*np.pi/180.
    dec2_rad=dec2*np.pi/180.

    del_ra=ra2_rad-ra1_rad
    del_ras=np.sin(del_ra)
    del_rac=np.cos(del_ra)
    dec1s=np.sin(dec1_rad)
    dec2s=np.sin(dec2_rad)
    dec1c=np.cos(dec1_rad)
    dec2c=np.cos(dec2_rad)
    arctan_arg=np.sqrt((dec2c**2)*(del_ras**2)+(dec1c*dec2s-dec1s*dec2c*del_rac)**2)/(dec1s*dec2s+dec1c*dec2c*del_rac)
    if arctan_arg < 0:
        arctan_arg = -arctan_arg
    return (180./np.pi)*np.arctan(arctan_arg)

class LineInfo(object):
    def __init__(self,dat_line,err_line):
        dat=dat_line.split()
        self.ion=dat[0]
        self.n=float(dat[1])
        self.b=float(dat[2])
        self.wav=float(dat[3])
        self.v=float(dat[4])
        self.n_flag=int(dat[5])
        self.b_flag=int(dat[6])
        self.v_flag=int(dat[7])
        self.eqw=float(dat[8])
        self.tot_eqw=float(dat[9])
        
        errs=err_line.split()
        self.n_err=float(errs[0])
        self.b_err=float(errs[1])
        self.v_err=float(errs[2])
        
    def __repr__(self):
        return self.ion
    def __str__(self):
        return self.ion

In [2]:
hdulist=fits.open('/DataDisk/datafiles/PGCC_HST/HFI_PCCS_GCC_R2.02.fits')
pgccs=hdulist[1].data

In [3]:
folders=[x for x in os.listdir(x1d_dir) if (os.path.isdir(x1d_dir+x) and x.startswith('.')==False)]                
ras=[]
decs=[]
nearest_pgccs=[]
for folder in folders:
    ra,dec=Simbad.Position(folder)
    ras.append(ra)
    decs.append(dec)
    nearest=100
    nearest_pgcc=[]
    for pgcc in pgccs:
        if abs(pgcc['ra']-ra.value)<1 and abs(pgcc['dec']-dec.value)<1:
            ang_sep=get_ang_sep(ra.value,dec.value,pgcc['ra'],pgcc['dec'])
            if ang_sep<1.0:
                nearest=ang_sep
                nearest_pgcc.append(pgcc)
    nearest_pgccs.append(nearest_pgcc)

In [53]:
coords=SkyCoord(ra=ras,dec=decs).galactic
ls=coords.l.value
bs=coords.b.value

In [54]:
sightlines=zip(folders,ls,bs,nearest_pgccs)

In [39]:
cfa_path='/DataDisk/datafiles/PGCC_HST/CO/Columbia-CfA/COGAL_all_raw.fits'
hdulist=fits.open(cfa_path)
cfa_data=hdulist[0].data
hdr=hdulist[0].header

In [61]:
cfa_data[np.isnan(cfa_data)]=0
vel_width=10.
vels=[hdr['CRVAL1']+i*hdr['CDELT1'] for i in range(hdr['NAXIS1'])]
glons=[hdr['CRVAL2']+i*hdr['CDELT2'] for i in range(hdr['NAXIS2'])]
#glons=[x if x>0.0 else x+360. for x in glons]
glats=[hdr['CRVAL3']+i*hdr['CDELT3'] for i in range(-hdr['NAXIS3']/2+1,hdr['NAXIS3']/2+1)]
good_data=['HD23180','HD24534','HD101436','HD114886','HD148937','HD152249','HDE303308','CPD-592603']
fl='test.pdf'
pdf=PdfPages(fl)
for sl in [x for x in sightlines if x[0] in good_data]:
    results_files=[x for x in os.listdir(x1d_dir+sl[0]+'/E140H') if x.startswith('co') and x.endswith('_results.txt')]
    if len(results_files)>0:
        f=open(x1d_dir+sl[0]+'/E140H/'+results_files[0])
        lines=[x.strip() for x in f.read().split('\n')]
        for i in range(len(lines)):
            if lines[i].startswith('CO'):
                line_data=LineInfo(lines[i],lines[i+1])
                helio_vel=line_data.v
                break
    else:
        helio_vel=0.
    
    cen_l=sl[1]
    if cen_l > 180:
        cen_l=cen_l-360
    cen_b=sl[2]
    icrs=SkyCoord(l=cen_l*u.degree,b=cen_b*u.degree, frame='galactic').icrs
    vel=helio_vel+HelioToLSR(icrs.ra.value,icrs.dec.value)

    min_vel=vel-vel_width
    max_vel=vel+vel_width
    min_vel_idx=min(range(len(vels)), key=lambda i: abs(vels[i]-min_vel))
    max_vel_idx=min(range(len(vels)), key=lambda i: abs(vels[i]-max_vel))

    # Isolate a box around the sightline
    box=2.0
    glon_idx=min(range(len(glons)),key=lambda i: abs(glons[i]-cen_l))
    glat_idx=min(range(len(glats)),key=lambda i: abs(glats[i]-cen_b))
    min_glon=min(range(len(glons)),key=lambda i: abs(glons[i]-(cen_l+box/2.)))
    max_glon=min(range(len(glons)),key=lambda i: abs(glons[i]-(cen_l-box/2.)))
    min_glat=min(range(len(glats)),key=lambda i: abs(glats[i]-(cen_b-box/2.)))
    max_glat=min(range(len(glats)),key=lambda i: abs(glats[i]-(cen_b+box/2.)))
    #print cfa_data[min_vel_idx:max_vel_idx,min_glon:max_glon,min_glat:max_glat]
    dat=np.sum(cfa_data[min_glat:max_glat,min_glon:max_glon,min_vel_idx:max_vel_idx],axis=2)
    dat=dat[::-1,:]

    fig=plt.figure(3,figsize=(12,9))
    fig.suptitle(sl[0],y=0.87,fontsize=24)
    G=gridspec.GridSpec(5,4)
    subplt1=plt.subplot(G[0:4,0:2])
    subplt2=plt.subplot(G[4,0:2])
    subplt3=plt.subplot(G[0:4,2:4])
    subplt4=plt.subplot(G[4,2:4])
    
    cax=subplt1.imshow(dat,extent=[glons[min_glon],glons[max_glon],glats[min_glat],glats[max_glat]])
    subplt1.plot(cen_l,cen_b, 'ro')
    subplt2.plot(vels,cfa_data[glat_idx,glon_idx,:])
    subplt2.set_title('Centered on CO Absorption velocity')
    if helio_vel!=0:
        subplt2.axvline(min_vel,0,1,linestyle='--',color='r')
        subplt2.axvline(max_vel,0,1,linestyle='--',color='r')
        
    vel_at_peak=vels[np.argmax(cfa_data[glat_idx,glon_idx,:])]
    min_vel=vel_at_peak-vel_width
    max_vel=vel_at_peak+vel_width
    min_vel_idx=min(range(len(vels)), key=lambda i: abs(vels[i]-min_vel))
    max_vel_idx=min(range(len(vels)), key=lambda i: abs(vels[i]-max_vel))
    dat=np.sum(cfa_data[min_glat:max_glat,min_glon:max_glon,min_vel_idx:max_vel_idx],axis=2)
    dat=dat[::-1,:]
    cax=subplt3.imshow(dat,extent=[glons[min_glon],glons[max_glon],glats[min_glat],glats[max_glat]])
    subplt3.plot(cen_l,cen_b, 'ro')
    subplt4.plot(vels,cfa_data[glat_idx,glon_idx,:])
    subplt4.set_title('Centered on CO Emission Peak')
    subplt4.axvline(min_vel,0,1,linestyle='--',color='r')
    subplt4.axvline(max_vel,0,1,linestyle='--',color='r')
    subplt3.set_yticklabels([])
    subplt4.set_yticklabels([])
    
    for pgcc in sl[3]:
        pgcc_glon=pgcc['glon']
        if pgcc_glon>180.:
            pgcc_glon=pgcc_glon-360.
        pgcc_glat=pgcc['glat']
        for i in range(1,4):
            ell=Ellipse(xy=(pgcc_glon,pgcc_glat),width=i*pgcc['gau_major_axis']/60.,
                        height=i*pgcc['gau_minor_axis']/60.,angle=(180./np.pi)*pgcc['gau_position_angle'])
            ell.set_facecolor('w')
            ell.set_alpha(0.6/i)
            subplt1.add_patch(ell)
            ell=Ellipse(xy=(pgcc_glon,pgcc_glat),width=i*pgcc['gau_major_axis']/60.,
                        height=i*pgcc['gau_minor_axis']/60.,angle=(180./np.pi)*pgcc['gau_position_angle'])
            ell.set_facecolor('w')
            ell.set_alpha(0.6/i)
            subplt3.add_patch(ell)
    if helio_vel!=0:
        pdf.savefig()
    plt.cla()
    plt.close()
pdf.close()